In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset

In [2]:
import zipfile

path = '/content/archive.zip'
zip_ref = zipfile.ZipFile(path,'r')
zip_ref.extractall() # or leave blank to extract to current directory

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
train_path = '/content/melanoma_cancer_dataset/train'
test_path = '/content/melanoma_cancer_dataset/test'

In [5]:
train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
])

test_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [6]:
# Lets now load our data using ImageFolder
train_data = ImageFolder(root=train_path,
                         transform= train_transforms)
test_data = ImageFolder(root=test_path,
                        transform= test_transforms)

print(f"Train data:\n{train_data}\nTest data:\n{test_data}")

Train data:
Dataset ImageFolder
    Number of datapoints: 9605
    Root location: /content/melanoma_cancer_dataset/train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
               RandomHorizontalFlip(p=0.5)
               RandomVerticalFlip(p=0.5)
               RandomRotation(degrees=[-15.0, 15.0], interpolation=nearest, expand=False, fill=0)
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 1000
    Root location: /content/melanoma_cancer_dataset/test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


In [7]:
import os
# Lets now turn our data into dataloaders
train_dataloader = DataLoader(dataset=train_data,
                              batch_size=32,
                              shuffle=True,
                              num_workers= os.cpu_count(),
                              )

test_dataloader = DataLoader(dataset=test_data,
                             batch_size=32,
                             shuffle = False,
                             num_workers = os.cpu_count(),
                             )

train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x7d31626cb550>,
 <torch.utils.data.dataloader.DataLoader at 0x7d321225eec0>)

In [8]:
# Lets get started with ViT (DynoV2)
# Clone the DINOv2 repository
!git clone https://github.com/facebookresearch/dinov2.git

# Change to the DINOv2 directory
%cd dinov2

# Install dependencies
!pip install -r requirements.txt


fatal: destination path 'dinov2' already exists and is not an empty directory.
/content/dinov2
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117, https://pypi.nvidia.com


In [9]:
import torch
from transformers import Dinov2ForImageClassification, AutoImageProcessor
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [10]:
# Load the pre-trained model and image processor
processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")
# No need processor, using own transformations
model = Dinov2ForImageClassification.from_pretrained("facebook/dinov2-base")
model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Dinov2ForImageClassification(
  (dinov2): Dinov2Model(
    (embeddings): Dinov2Embeddings(
      (patch_embeddings): Dinov2PatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Dinov2Encoder(
      (layer): ModuleList(
        (0-11): 12 x Dinov2Layer(
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (attention): Dinov2Attention(
            (attention): Dinov2SelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): Dinov2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
       

In [11]:
!pip install torchinfo
from torchinfo import summary
summary(model,
        input_size=(32, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                           Input Shape          Output Shape         Param #              Trainable
Dinov2ForImageClassification (Dinov2ForImageClassification)       [32, 3, 224, 224]    [32, 2]              --                   True
├─Dinov2Model (dinov2)                                            [32, 3, 224, 224]    [32, 768]            --                   True
│    └─Dinov2Embeddings (embeddings)                              [32, 3, 224, 224]    [32, 257, 768]       1,053,696            True
│    │    └─Dinov2PatchEmbeddings (patch_embeddings)              [32, 3, 224, 224]    [32, 256, 768]       452,352              True
│    │    └─Dropout (dropout)                                     [32, 257, 768]       [32, 257, 768]       --                   --
│    └─Dinov2Encoder (encoder)                                    [32, 257, 768]       [32, 257, 768]       --                   True
│    │    └─ModuleList (layer)                             

In [12]:
# Assume `model` is your Dinov2ForImageClassification instance
for param in model.dinov2.parameters():
    param.requires_grad = False  # Freeze all parameters in the Dinov2 model

# Unfreeze the classifier
for param in model.classifier.parameters():
    param.requires_grad = True  # Train the classifier parameters


In [13]:
!pip install torchinfo
from torchinfo import summary
summary(model,
        input_size=(32, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                           Input Shape          Output Shape         Param #              Trainable
Dinov2ForImageClassification (Dinov2ForImageClassification)       [32, 3, 224, 224]    [32, 2]              --                   Partial
├─Dinov2Model (dinov2)                                            [32, 3, 224, 224]    [32, 768]            --                   False
│    └─Dinov2Embeddings (embeddings)                              [32, 3, 224, 224]    [32, 257, 768]       1,053,696            False
│    │    └─Dinov2PatchEmbeddings (patch_embeddings)              [32, 3, 224, 224]    [32, 256, 768]       (452,352)            False
│    │    └─Dropout (dropout)                                     [32, 257, 768]       [32, 257, 768]       --                   --
│    └─Dinov2Encoder (encoder)                                    [32, 257, 768]       [32, 257, 768]       --                   False
│    │    └─ModuleList (layer)                      

In [14]:
# Lets define a loss function and a optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [15]:
import torch
from tqdm import tqdm

# Move the model to the appropriate device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 10

for epoch in tqdm(range(num_epochs)):
    model.train()
    running_loss = 0
    running_corrects = 0  # Initialize correct predictions counter
    total_samples = 0  # Initialize total samples counter

    for batch_idx, (images, labels) in enumerate(train_dataloader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images).logits  # Access logits directly
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate predictions and accuracy
        _, preds = torch.max(outputs, 1)  # Get predicted class indices
        running_corrects += (preds == labels).sum().item()  # Count correct predictions
        total_samples += labels.size(0)  # Count total samples in the batch

        # Print every 10 batches
        if batch_idx % 10 == 0:
            batch_accuracy = (preds == labels).float().mean()  # Calculate batch accuracy
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx}/{len(train_dataloader)}], "
                  f"Loss: {loss.item():.4f}, Batch Accuracy: {batch_accuracy:.4f}")

    # Calculate and print epoch-level metrics
    epoch_accuracy = running_corrects / total_samples  # Calculate accuracy for the epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {running_loss/len(train_dataloader):.4f}, "
          f"Epoch Accuracy: {epoch_accuracy:.4f}")

# Save the model state after training
torch.save(model.state_dict(), 'dinov2_finetuned.pth')


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch [1/10], Batch [0/301], Loss: 1.1903, Batch Accuracy: 0.2500
Epoch [1/10], Batch [10/301], Loss: 0.7014, Batch Accuracy: 0.5000
Epoch [1/10], Batch [20/301], Loss: 0.6690, Batch Accuracy: 0.6875
Epoch [1/10], Batch [30/301], Loss: 0.4188, Batch Accuracy: 0.8438
Epoch [1/10], Batch [40/301], Loss: 0.5326, Batch Accuracy: 0.7188
Epoch [1/10], Batch [50/301], Loss: 0.4631, Batch Accuracy: 0.8125
Epoch [1/10], Batch [60/301], Loss: 0.3481, Batch Accuracy: 0.8438
Epoch [1/10], Batch [70/301], Loss: 0.4884, Batch Accuracy: 0.7188
Epoch [1/10], Batch [80/301], Loss: 0.3578, Batch Accuracy: 0.9062
Epoch [1/10], Batch [90/301], Loss: 0.3825, Batch Accuracy: 0.7812
Epoch [1/10], Batch [100/301], Loss: 0.4786, Batch Accuracy: 0.7812
Epoch [1/10], Batch [110/301], Loss: 0.4226, Batch Accuracy: 0.8125
Epoch [1/10], Batch [120/301], Loss: 0.2466, Batch Accuracy: 0.9375
Epoch [1/10], Batch [130/301], Loss: 0.3785, Batch Accuracy: 0.8125
Epoch [1/10], Batch [140/301], Loss: 0.2823, Batch Accuracy

 10%|█         | 1/10 [02:29<22:26, 149.58s/it]

Epoch [1/10], Batch [300/301], Loss: 0.1214, Batch Accuracy: 1.0000
Epoch [1/10], Average Loss: 0.3625, Epoch Accuracy: 0.8327
Epoch [2/10], Batch [0/301], Loss: 0.3809, Batch Accuracy: 0.8750
Epoch [2/10], Batch [10/301], Loss: 0.1835, Batch Accuracy: 0.9688
Epoch [2/10], Batch [20/301], Loss: 0.4525, Batch Accuracy: 0.8438
Epoch [2/10], Batch [30/301], Loss: 0.3941, Batch Accuracy: 0.8438
Epoch [2/10], Batch [40/301], Loss: 0.2322, Batch Accuracy: 0.8438
Epoch [2/10], Batch [50/301], Loss: 0.3025, Batch Accuracy: 0.9062
Epoch [2/10], Batch [60/301], Loss: 0.2940, Batch Accuracy: 0.9062
Epoch [2/10], Batch [70/301], Loss: 0.2055, Batch Accuracy: 0.9688
Epoch [2/10], Batch [80/301], Loss: 0.1541, Batch Accuracy: 0.9688
Epoch [2/10], Batch [90/301], Loss: 0.2298, Batch Accuracy: 0.9688
Epoch [2/10], Batch [100/301], Loss: 0.3921, Batch Accuracy: 0.8125
Epoch [2/10], Batch [110/301], Loss: 0.3561, Batch Accuracy: 0.8750
Epoch [2/10], Batch [120/301], Loss: 0.2301, Batch Accuracy: 0.8750


 20%|██        | 2/10 [04:59<19:59, 149.91s/it]

...

Epoch [8/10], Batch [300/301], Loss: 0.0835, Batch Accuracy: 1.0000
Epoch [8/10], Average Loss: 0.1990, Epoch Accuracy: 0.9206
Epoch [9/10], Batch [0/301], Loss: 0.2003, Batch Accuracy: 0.9062
Epoch [9/10], Batch [10/301], Loss: 0.2463, Batch Accuracy: 0.8750
Epoch [9/10], Batch [20/301], Loss: 0.1817, Batch Accuracy: 0.9375
Epoch [9/10], Batch [30/301], Loss: 0.1414, Batch Accuracy: 0.9688
Epoch [9/10], Batch [40/301], Loss: 0.2197, Batch Accuracy: 0.9062
Epoch [9/10], Batch [50/301], Loss: 0.2479, Batch Accuracy: 0.9062
Epoch [9/10], Batch [60/301], Loss: 0.3166, Batch Accuracy: 0.9062
Epoch [9/10], Batch [70/301], Loss: 0.1456, Batch Accuracy: 0.9375
Epoch [9/10], Batch [80/301], Loss: 0.2557, Batch Accuracy: 0.8438
Epoch [9/10], Batch [90/301], Loss: 0.1240, Batch Accuracy: 0.9688
Epoch [9/10], Batch [100/301], Loss: 0.3617, Batch Accuracy: 0.8750
Epoch [9/10], Batch [110/301], Loss: 0.1355, Batch Accuracy: 0.9688
Epoch [9/10], Batch [120/301], Loss: 0.3203, Batch Accuracy: 0.9062


 90%|█████████ | 9/10 [22:29<02:29, 149.98s/it]

Epoch [9/10], Batch [300/301], Loss: 0.0338, Batch Accuracy: 1.0000
Epoch [9/10], Average Loss: 0.1956, Epoch Accuracy: 0.9231
Epoch [10/10], Batch [0/301], Loss: 0.3657, Batch Accuracy: 0.8750
Epoch [10/10], Batch [10/301], Loss: 0.4371, Batch Accuracy: 0.7500
Epoch [10/10], Batch [20/301], Loss: 0.2025, Batch Accuracy: 0.9375
Epoch [10/10], Batch [30/301], Loss: 0.1615, Batch Accuracy: 0.9375
Epoch [10/10], Batch [40/301], Loss: 0.2651, Batch Accuracy: 0.9062
Epoch [10/10], Batch [50/301], Loss: 0.2407, Batch Accuracy: 0.8750
Epoch [10/10], Batch [60/301], Loss: 0.2093, Batch Accuracy: 0.9375
Epoch [10/10], Batch [70/301], Loss: 0.2246, Batch Accuracy: 0.8750
Epoch [10/10], Batch [80/301], Loss: 0.2157, Batch Accuracy: 0.8750
Epoch [10/10], Batch [90/301], Loss: 0.3437, Batch Accuracy: 0.8750
Epoch [10/10], Batch [100/301], Loss: 0.0793, Batch Accuracy: 1.0000
Epoch [10/10], Batch [110/301], Loss: 0.1952, Batch Accuracy: 0.9375
Epoch [10/10], Batch [120/301], Loss: 0.2146, Batch Accu

100%|██████████| 10/10 [24:59<00:00, 149.94s/it]

Epoch [10/10], Batch [300/301], Loss: 0.1885, Batch Accuracy: 1.0000
Epoch [10/10], Average Loss: 0.1950, Epoch Accuracy: 0.9237


In [16]:
import torch
from tqdm import tqdm
import time  # Import time module for latency measurement
import subprocess  # Import subprocess to call nvidia-smi for GPU usage

# Move the model to the appropriate device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Set the model to evaluation mode
model.eval()

running_loss = 0
running_corrects = 0
total_samples = 0
total_latency = 0  # Initialize total latency
gpu_memory_list = []  # List to track GPU memory usage per batch
gpu_usage_list = []  # List to track GPU utilization per batch

# Disable gradient calculation for testing
with torch.inference_mode():
    for batch_idx, (images, labels) in enumerate(tqdm(test_dataloader)):
        start_time = time.time()  # Start time for latency measurement

        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images).logits  # Access logits directly
        loss = loss_fn(outputs, labels)

        # Update running loss
        running_loss += loss.item()

        # Calculate predictions and accuracy
        _, preds = torch.max(outputs, 1)  # Get predicted class indices
        running_corrects += (preds == labels).sum().item()  # Count correct predictions
        total_samples += labels.size(0)  # Count total samples in the batch

        # Calculate latency for this batch
        batch_latency = time.time() - start_time
        total_latency += batch_latency  # Update total latency

        # Capture GPU memory usage (in MB)
        if device.type == 'cuda':
            gpu_memory = torch.cuda.memory_allocated() / (1024 ** 2)  # Convert bytes to MB
            gpu_memory_list.append(gpu_memory)

            # Capture real-time GPU usage with nvidia-smi
            result = subprocess.run(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'],
                                    stdout=subprocess.PIPE, text=True)
            gpu_usage = float(result.stdout.strip())  # Extract GPU usage percentage
            gpu_usage_list.append(gpu_usage)

        # Print every 10 batches
        if batch_idx % 10 == 0:
            batch_accuracy = (preds == labels).float().mean()  # Calculate batch accuracy
            print(f"Test Batch [{batch_idx}/{len(test_dataloader)}], "
                  f"Loss: {loss.item():.4f}, Batch Accuracy: {batch_accuracy:.4f}, "
                  f"Latency: {batch_latency:.4f} seconds")

# Calculate and print epoch-level metrics
test_accuracy = running_corrects / total_samples  # Calculate accuracy for the test set
average_loss = running_loss / len(test_dataloader)  # Calculate average loss
average_latency = total_latency / len(test_dataloader)  # Calculate average latency per batch

# Calculate average GPU memory usage
average_gpu_memory = sum(gpu_memory_list) / len(gpu_memory_list) if gpu_memory_list else 0
average_gpu_usage = sum(gpu_usage_list) / len(gpu_usage_list) if gpu_usage_list else 0

# Print final metrics
print(f'Test Loss: {average_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')  # Convert accuracy to percentage
print(f'Average Latency per Batch: {average_latency:.4f} seconds')
if device.type == 'cuda':
    print(f'Average GPU Memory Usage: {average_gpu_memory:.2f} MB')
    print(f'Average GPU Utilization: {average_gpu_usage:.2f}%')
    print(f'Peak GPU Memory Usage: {torch.cuda.max_memory_allocated() / (1024 ** 2):.2f} MB')


  3%|▎         | 1/32 [00:00<00:29,  1.04it/s]

Test Batch [0/32], Loss: 0.2865, Batch Accuracy: 0.9062, Latency: 0.5393 seconds


 34%|███▍      | 11/32 [00:06<00:11,  1.88it/s]

Test Batch [10/32], Loss: 0.1823, Batch Accuracy: 0.9688, Latency: 0.4780 seconds


 66%|██████▌   | 21/32 [00:11<00:05,  1.89it/s]

Test Batch [20/32], Loss: 0.0011, Batch Accuracy: 1.0000, Latency: 0.4919 seconds


100%|██████████| 32/32 [00:17<00:00,  2.37it/s]

Test Batch [30/32], Loss: 0.8548, Batch Accuracy: 0.5938, Latency: 0.4824 seconds


100%|██████████| 32/32 [00:17<00:00,  1.86it/s]

Test Loss: 0.2402
Test Accuracy: 91.30%
Average Latency per Batch: 0.4731 seconds
Average GPU Memory Usage: 365.86 MB
Average GPU Utilization: 99.00%
Peak GPU Memory Usage: 770.35 MB


In [ ]:
from google.colab import files

# Specify the path to the file you want to download
file_path = '/content/dinov2/dinov2/dinov2_finetuned.pth'

# Download the file
files.download(file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>